In [1]:
import os
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import nltk
import random
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import pickle

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def stem_words(doc):
    stemmed_doc = ''
    for w in doc.split(' '):
        stemmed_doc += stemmer.stem(w) + ' '
    return stemmed_doc

def train(data, iterations):
    models = []
    corpus = data['corpus']
    labels = data['labels']
    for i in range(iterations):
        random_indicies = random.sample(data['c_w_corpus'], 500) +random.sample(data['piper_corpus'],500) +random.sample(data['spurgeon_corpus'], 500) +random.sample(data['tozer_corpus'], 500)
        X = corpus[random_indicies,:]
        y = [labels[i] for i in random_indicies]
        print('Generating', i+1)
        if i % 2 ==0:
            result = generate_SVC_clasifier_wo_vect(X,y)
        else:
            result = generate_tree_clasifier(X,y)
        models.append(result)
    with open('models.pkl', 'wb') as f:
        pickle.dump(models, f)
        
def predict(X, y, vect):
    models =pickle.load( open( "models.pkl", "rb" ) )
    preds = []
    final_prediction = []
    print('Vectorizing Input')
    X = vect.transform(X)
    print('Predicting')
    for m in models: 
        preds.append(m.predict(X))
    for i in range(len(preds[0])):
        this_pred = []
        for j in range(len(preds)):
            this_pred.append(preds[j][i])
        final_prediction.append(most_common(this_pred))
    print("Multinomial NB: " , round(accuracy_score(y, final_prediction), 3))
    print(classification_report(y, final_prediction, labels=range(0,4)))
    
def generate_SVC_clasifier_wo_vect(X, y):
    print('Fitting SVC')
    svc = SVC()#, class_weight = weight_dict)
    svc.fit(X, y)
    return svc

def generate_tree_clasifier(X, y):
    print('Fitting Tree')
    tree = DecisionTreeClassifier(max_depth = 10)
    tree.fit(X, y)
    return tree

def most_common(lst):
    return max(set(lst), key=lst.count)

def load_sermons():
    data = []
    labels = []
    label= 0
    count=0
    authors = ['charles_wesley', 'spurgeon', 'tozer', 'piper']
    for author in authors:
        for sermon in os.listdir('./sermons/' + author):   
            #os.remove('./sermons/' + author + '/.DS_Store')
            f=open('./sermons/'+ author + '/' + sermon, "r")
            if f.mode == 'r':
                contents =f.read()
                if len(contents) > 100:
                    contents = stem_words(contents)
                    data.append(contents)
                    labels.append(label)
            count +=1
        print(label, author)
        label +=1
        count = 0
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.23)
    c_w = [] 
    piper = [] 
    spurgeon=[] 
    tozer = [] 

    for i in range(len(y_train)):
        if y_train[i] == 0:
            c_w.append((X_train[i], 0))
        elif y_train[i] == 1:
            spurgeon.append((X_train[i], 1))
        elif y_train[i] == 2:
            tozer.append((X_train[i], 2))
        elif y_train[i] == 3:
            piper.append((X_train[i], 3))
            
    X = random.sample(c_w, 500) +random.sample(piper,500) +random.sample(spurgeon, 500) +random.sample(tozer, 500)+ random.sample(c_w, 500) +random.sample(piper,500) +random.sample(spurgeon, 500) +random.sample(tozer, 500)
    np.random.shuffle(X)
    X, y = [item[0] for item in X], [item[1] for item in X]
    vect = TfidfVectorizer(stop_words = "english", ngram_range = {1,4})
    corpus = vect.fit_transform(X)

    c_w_corpus = [] 
    piper_corpus = [] 
    spurgeon_corpus =[] 
    tozer_corpus = [] 

    for i in range(len(y)):
        if y[i] == 0:
            c_w_corpus.append(i)
        elif y[i] == 1:
            spurgeon_corpus.append(i)
        elif y[i] == 2:
            tozer_corpus.append(i)
        elif y[i] == 3:
            piper_corpus.append(i)
    return {'c_w_corpus':c_w_corpus, 'piper_corpus':piper_corpus, 
            'spurgeon_corpus':spurgeon_corpus, 'tozer_corpus':tozer_corpus, 'corpus':corpus, 
            'X_test':X_test, 'y_test':y_test, 'labels':y, 'vect': vect}

In [3]:
data = load_sermons()
models = train(data, 10)
predict(data['X_test'], data['y_test'], data['vect'])

0 charles_wesley
1 spurgeon
2 tozer
3 piper
Generating 1
Fitting SVC
Generating 2
Fitting Tree
Generating 3
Fitting SVC
Generating 4
Fitting Tree
Generating 5
Fitting SVC
Generating 6
Fitting Tree
Generating 7
Fitting SVC
Generating 8
Fitting Tree
Generating 9
Fitting SVC
Generating 10
Fitting Tree
Vectorizing Input
Predicting
Multinomial NB:  0.873
             precision    recall  f1-score   support

          0       0.80      0.99      0.89       177
          1       0.92      0.61      0.74       153
          2       0.74      0.95      0.83       207
          3       0.98      0.88      0.93       455

avg / total       0.89      0.87      0.87       992



In [11]:
f=open('./sermons/john_wesley/john_wesley_Duty_of_Constant_Communion.txt', "r")
if f.mode == 'r':
    contents = [f.read() ]

kyle = vect.transform(contents)
prob = nb.predict(kyle)
prob
#for i in range(0,4):
    #print(authors[i], " : " , prob[0][i]*100 , "%")


array([1])